In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch import Tensor
print(torch.__version__)

2.3.0+cu121


In [2]:
import os
os.environ['TORCH'] = torch.__version__

# Siguiendo
https://github.com/pyg-team/pytorch_geometric/blob/master/examples/link_pred.py

In [3]:
#Get the current working directory
cwd = os.getcwd()
file_path = cwd + '/points_50.csv'

In [4]:
# Importing the dataset
df = pd.read_csv(file_path)

In [5]:
#Round the values of the dataset to 4 decimal places
df = df.round(4)

#Add a column to use as index from 0 to the length of the dataset
df['n_label'] = range(0, len(df))

#delete the column p_label
df = df.drop('p_label', axis=1)

In [6]:
df.head()

,x,y,z,N_side,N_layer,t_label,phi,eta,q,pt,d0,z0,n_label
0,-0.1120,-9.9682,-6.3331,8,1,T0,-1.6049,-0.6008,-1,48.2712,0.2156,0.022,0
1,-0.4144,-19.8918,-12.6639,8,2,T0,-1.6049,-0.6008,-1,48.2712,0.2156,0.022,1
2,-0.6930,-29.8162,-18.9948,8,3,T0,-1.6049,-0.6008,-1,48.2712,0.2156,0.022,2
3,-0.9483,-39.7538,-25.3337,8,4,T0,-1.6049,-0.6008,-1,48.2712,0.2156,0.022,3
4,-1.1794,-49.6794,-31.6646,8,5,T0,-1.6049,-0.6008,-1,48.2712,0.2156,0.022,4


In [7]:
#Create a pytorch geometric data object
from torch_geometric.data import Data

data=Data()


In [8]:
#Add nodes to the data object from n_label column
data.x = torch.tensor(df[['x','y','z','phi','eta','pt','d0','z0']].values, dtype=torch.float)

In [9]:
edge_path = cwd + '/grap_50.csv'

# Importing the dataset
df_edge = pd.read_csv(edge_path)

In [10]:
#Add edges to the data object
data.edge_index = torch.tensor(df_edge[['Source','Target']].values, dtype=torch.long).t().contiguous()

In [11]:
#data.edge_attr = torch.tensor(df_edge['weight'], dtype=torch.float).view(-1, 1)

In [12]:
data.validate(raise_on_error=True)

True

In [13]:
import torch_geometric.transforms as T
data = T.ToUndirected()(data)

In [14]:
#Save the data object
#torch.save(data, 'data_hom.pt')

In [15]:
data

Data(x=[500, 8], edge_index=[2, 1384])

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True,
                      add_negative_train_samples=False),
])

In [18]:
train_data, val_data, test_data = transform(data)

In [19]:
print(train_data)
print("...................")
print(val_data)
print("...................")
print(test_data)

Data(x=[500, 8], edge_index=[2, 1178], edge_label=[589], edge_label_index=[2, 589])
...................
Data(x=[500, 8], edge_index=[2, 1178], edge_label=[68], edge_label_index=[2, 68])
...................
Data(x=[500, 8], edge_index=[2, 1246], edge_label=[138], edge_label_index=[2, 138])


In [20]:
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling

class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

In [21]:
model = Net(data.num_features, 128, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

In [22]:
print(model)

Net(
  (conv1): GCNConv(8, 128)
  (conv2): GCNConv(128, 64)
)


In [23]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss

In [24]:
from sklearn.metrics import roc_auc_score

@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())

In [25]:
best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

print(f'Final Test: {final_test_auc:.4f}')

Epoch: 001, Loss: 0.6929, Val: 0.5277, Test: 0.3573
Epoch: 002, Loss: 0.6927, Val: 0.4913, Test: 0.3226
Epoch: 003, Loss: 0.6932, Val: 0.5285, Test: 0.3621
Epoch: 004, Loss: 0.6921, Val: 0.5510, Test: 0.3886
Epoch: 005, Loss: 0.6919, Val: 0.5476, Test: 0.3886
Epoch: 006, Loss: 0.6915, Val: 0.5407, Test: 0.3823
Epoch: 007, Loss: 0.6910, Val: 0.5536, Test: 0.3955
Epoch: 008, Loss: 0.6907, Val: 0.5796, Test: 0.4304
Epoch: 009, Loss: 0.6899, Val: 0.5865, Test: 0.4476
Epoch: 010, Loss: 0.6885, Val: 0.5718, Test: 0.4211
Epoch: 011, Loss: 0.6878, Val: 0.6003, Test: 0.4791
Epoch: 012, Loss: 0.6870, Val: 0.6808, Test: 0.7441
Epoch: 013, Loss: 0.6855, Val: 0.6765, Test: 0.6912
Epoch: 014, Loss: 0.6823, Val: 0.6081, Test: 0.4829
Epoch: 015, Loss: 0.6797, Val: 0.6176, Test: 0.5201
Epoch: 016, Loss: 0.6789, Val: 0.6332, Test: 0.7236
Epoch: 017, Loss: 0.6752, Val: 0.7024, Test: 0.7633
Epoch: 018, Loss: 0.6717, Val: 0.6410, Test: 0.5528
Epoch: 019, Loss: 0.6673, Val: 0.6479, Test: 0.5887
Epoch: 020, 

In [26]:
z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode_all(z)